# Notebook 4: Shoreline Derivation


_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_


A shoreline is the interface between the land and the ocean. The location of the shoreline is fundamentally important to various aspects of coastal science and engineering. However, the cross-shore and vertical location of the shoreline continuously changes both temporally and spatially. Thus, differing definitions of the shoreline exist for various applications.Shoreline location is often approximated by a constant elevation contour drawn in a tidal datum. The specific tidal datum and contour elevation may depend on the application. The following figure shows the shoreline migration range in the town of Nags Head (image from Hardin et al (2014) ).

![](img/shoreline.png)

In this notebook we will:
* Derive shorelines using NAGS Head dataset
* Smoothning of extracted shorelines 
* Visualization of the extracted shorelines

***

## 1. Import Python Packages

In [1]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj



***

## 2. Start GRASS Session

In [2]:
gj.init("./nags_head/PERMANENT");

***

## 3. Deriving and smoothing extracted shorelines

In [3]:
# Purpose: Use r.contour to extract contour.
# l is the shoreline proxy elevation
# c is the threshold to exclude small anomalies
l = 0.8
c = 400
dates = ['2014', '2016', '2018','2020']

for date in dates:
    elevation = f'JR_{date}'
    contour=f'JR_{date}_contour'
    shoreline=f'JR_{date}_shoreline'
    
    gs.run_command( 'r.contour', input=elevation, output=contour, levels=l, cut=c)
    gs.run_command('v.generalize',input=contour, output=shoreline, method='sliding_averaging', threshold=0, look_ahead=51)

***

## 4. Visualization

In [4]:
shoreline_series = gs.read_command("g.list", type="vector", pattern="*shoreline").split()
shoreline_series

['JR_2014_shoreline',
 'JR_2016_shoreline',
 'JR_2018_shoreline',
 'JR_2020_shoreline']

In [5]:
img = gj.Map()
# img.d_rgb(naip)
img=gj.InteractiveMap(width=800, tiles="OpenStreetMap")
for shoreline in shoreline_series:
    img.add_vector(shoreline)
img.add_layer_control()
img.show()